In [85]:
# Import the various libraries
import pandas as pd
import os
from SPARQLWrapper import SPARQLWrapper, JSON

In [86]:
# Settings for files
folderlink = '..//data//'
input_folder = 'input//'
folder_output = 'output//'

In [87]:
# Add sparql endpoint
sparql_endpoint = "http://data.bibliotheken.nl/sparql"

In [88]:
# In order to link the individuals from CLERUS with book titles, this script allows to generate a datadump of booktitle metadata from the Koninklijke Bibliotheek. The dataset that is of interest to connect CLERUS with is the "Short Title Catalogue Netherlands" STCN. 
# At the request of the Lead Applicant of the project the following fields have been identified as interesting. The LA requested an excelsheet with one unique row per booktiltle. Since, not the dataset contains many one to many relationships, every field has been put into one SPARQL query and in case of multiple childs these have been put in the same field. 

# Below an overview on the various Queries
# q1 the primary key i.e. the recordnummers
# q2 taal - schema:inLanguage 
# q3 jaar - schema:publication/schema:startDate 
# q4 land - schema:publication/schema:location
# q1 titelprimair - schema:name
# q5 titelsecundair - schema:alternateName
# q5 titelextra - schema:alternateName
# q6 auteurpersoon - schema:author/schema:name
# q7 auteursecundair - schema:contributor/schema:name
# q8 drukkervermelding - schema:publication/schema:description
# q9 drukkerpersoon - schema:publication/schema:publishedBy/schema:name
# q10 formaat - <http://data.bibliotheken.nl/def#bibliographicFormat> 
# q11 trefwoord - schema:about/skos:label

In [89]:
# The current script generates a dump for the STCN dataset with the fieldnames requested by the the Lead applicant from the project.
# In order to generate a similar datadump with the same fields for the Nederlandse Bibliografie Totaal (NBT) "stcn" should be changed into "nbt"

dataset_kb = "nbt"

In [90]:
# Get a list with all the records and a title
value = str(10)
primary_key = """
SELECT ?title_id ?title WHERE {
?title_id a schema:Book .
?title_id schema:name ?title .
?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
}
LIMIT """+value+""" 
"""

In [91]:
sparql = SPARQLWrapper(sparql_endpoint)
sparql.setQuery(primary_key)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
bindings = results["results"]["bindings"]
data = []
for item in bindings:
    row = {}
    for key in item:
        row[key] = item[key]["value"]
    data.append(row)
df = pd.DataFrame(data)

In [92]:
df.head()

,title_id,title
0,http://data.bibliotheken.nl/id/nbt/p750005645,Nonarchimedean fields and asymptotic expansions
1,http://data.bibliotheken.nl/id/nbt/p750009543,Urethral obstruction in boys : diagnosis and t...
2,http://data.bibliotheken.nl/id/nbt/p750009969,Health and industrial growth
3,http://data.bibliotheken.nl/id/nbt/p750012978,Good news in Acts : the ''Acts of the Apostles...
4,http://data.bibliotheken.nl/id/nbt/p750023635,Basisvorming voor alle leerlingen? : de werksi...


In [93]:
# Get a list with all the records and languages if present (can be more then one per item)
query2 = {"language":
"""
select ?title_id ?language where {
  ?title_id schema:inLanguage ?language .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10
"""}

In [94]:
# Get a list with all the records and a title and the year of publication if present
query3 = {"pub_year":
"""
select ?title_id ?pub_year where {
  ?title_id schema:publication/schema:startDate ?pub_year .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10
"""}

In [95]:
# Get a list with all the records and a title and the publication location if present
query4 = {"pub_location":
"""
select ?title_id ?pub_location {
  ?title_id schema:publication/schema:location ?pub_location .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10
"""}

In [96]:
# Get a list with all the records and an alternative title if present
query5 = {"alt_title":
"""
select ?title_id ?alt_title where {
  ?title_id schema:alternateName ?alt_title .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10
"""}

In [97]:
# Get a list with all the records and the main author 
query6 = {"author":
"""
select ?title_id ?author ?authorid where {
  ?title_id schema:author/schema:name ?author .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .  
  }
  LIMIT 10

"""}

In [98]:
# Get a list with all the records and if present the co author(s)
query7 = {"co_author":
"""
select ?title_id ?co_author where {
  ?title_id schema:contributor/schema:name ?co_author.
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10  

"""}

In [99]:
# Get a list with all the records and additional infor about the publishing entity
query8 = {"pub_description":
"""
select ?title_id ?pub_description where {
  ?title_id schema:publication/schema:description ?pub_description .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10  

"""}

In [100]:
# Get a list with all the records and the name of the publisher
query9 = {"publisher_name":
"""
select ?title_id ?publisher_name where {
  ?title_id schema:publication/schema:publishedBy/schema:name ?publisher_name .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10

"""}

In [101]:
# Get a list with all the records and the format of the publication
query10 = {"format":
"""
select ?title_id ?format where {
  ?title_id <http://data.bibliotheken.nl/def#bibliographicFormat> ?format .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10

"""}

In [102]:
# Get a list with all the records and the keywords that are assigned to them. 
query11 = {"keywords":
"""
select ?title_id ?keywords where {
  ?title_id schema:about/skos:label ?keywords .
  ?title_id a schema:Book .
  ?title_id schema:name ?title .
  ?title_id schema:mainEntityOfPage/schema:isPartOf <http://data.bibliotheken.nl/id/dataset/"""+dataset_kb+"""> .
  }
  LIMIT 10
"""}

In [103]:
query_dict = {**query2, **query3, **query4, **query6, **query7}#, **query9, **query10, **query11}

In [104]:
for name, query in query_dict.items():
    sparql = SPARQLWrapper(sparql_endpoint)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    bindings = results["results"]["bindings"]
    data = []
    for item in bindings:
        row = {}
        for key in item:
            row[key] = item[key]["value"]
        data.append(row)

    df_data = pd.DataFrame(data)
    
    result = df_data.groupby('title_id')[name].apply('; '.join).reset_index()
    df_join = pd.merge(df, result, left_on='title_id', right_on='title_id', how='left')
    df = df_join
    

In [109]:
df.head()

,title_id,title,language,pub_year,pub_location,author,co_author
0,http://data.bibliotheken.nl/id/nbt/p750005645,Nonarchimedean fields and asymptotic expansions,en,1975,nodeID://b297768423,NaN,NaN
1,http://data.bibliotheken.nl/id/nbt/p750009543,Urethral obstruction in boys : diagnosis and treatment of congenital valves of the posterior urethra,en,1975,nodeID://b297768444,NaN,NaN
2,http://data.bibliotheken.nl/id/nbt/p750009969,Health and industrial growth,en,1975,nodeID://b297768449,NaN,NaN
3,http://data.bibliotheken.nl/id/nbt/p750012978,Good news in Acts : the ''Acts of the Apostles'' in today's English version,nl,1974,nodeID://b297768454,NaN,NaN
4,http://data.bibliotheken.nl/id/nbt/p750023635,Basisvorming voor alle leerlingen? : de werksituatie van LBO-leerkrachten in het perspektief van basisvorming,nl,1987,nodeID://b297768467,NaN,NaN


In [106]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [107]:
df_join.to_csv(folderlink+folder_output+dataset_kb+"_Dump.csv", sep=';', encoding='utf-8', index=False)

In [108]:
excel_export = folderlink+folder_output+dataset_kb+"excel.xlsx"
df_join.to_excel(excel_export, index=False, engine='openpyxl')
